In [7]:
# load ztf data
from pathlib import Path
from datetime import datetime
import pandas as pd
import lasair
import numpy as np
from utils import processLasairData

project_root = Path.cwd().parent
folder_name = input("Enter name for run folder (leave blank for timestamp): ").strip()
if not folder_name:
    start_time = datetime.now()
    folder_name = start_time.strftime("%Y-%m-%d_%H-%M-%S")
run_folder = project_root / "runs" / folder_name
run_folder.mkdir(parents=True, exist_ok=True)

ztf_data = project_root / "ztf_cleansed.csv"

ztf_df = pd.read_csv(ztf_data)

print(ztf_df.head())

Enter name for run folder (leave blank for timestamp): test2
          ZTFID      IAUID           RA          Dec     peakt peakfilt  \
0  ZTF17aabtvsy  SN2022yei  10:35:32.09  +37:38:59.0  59870.49        r   
1  ZTF17aabvong  SN2024xxq  02:05:07.68  +11:14:55.1  60606.25        g   
2  ZTF17aacldgo  SN2022zxv  03:09:24.36  -04:53:39.2  59897.25        g   
3  ZTF17aadlxmv  SN2020adv  08:29:47.59  +33:54:22.8  58879.19        g   
4  ZTF18aaaonon  SN2022jjs  10:19:05.51  +14:24:16.6  59703.27        g   

   peakmag  peakabs duration    rise      fade   type  redshift          b  \
0  18.0303   -19.41  >34.229   >6.01    28.219  SN Ia   0.06922  59.641962   
1  16.8039   -19.57   23.222   8.464    14.758  SN Ia   0.03400 -47.664064   
2  18.7979   -18.91    >1077   >3.85  >1073.15  SN Ia   0.07200 -50.332472   
3  17.9475   -19.34   25.146  10.951    14.195  SN Ia   0.06200  34.174702   
4  18.5663   -19.08  >18.631   3.691    >14.94  SN Ia   0.07141  52.363911   

     A_V  host_reds

In [ ]:
from typing import Any

lasair_token = 'c4ce6509f05363cfb57aaedffb056ef31573e647'
client = lasair.lasair_client(lasair_token)
print("Lasair client initialized.")

num_total = len(ztf_df)
num_processed = 0
num_success = 0

for idx, row in enumerate[Any](ztf_df.itertuples(index=False)):
    num_processed += 1
    print(f"\nProcessing {num_processed:3d}/{num_total}: {row.ZTFID}...", end='')
    ztf_id = row.ZTFID
    iauid = row.IAUID
    if str(iauid).startswith('SN'):
        iauid = str(iauid)[2:]

    try:
        result = client.object(ztf_id)
    except Exception as e:
        print(f"\nLasair API error for {ztf_id}: {e}")
        continue

    if not result or 'candidates' not in result:
        print(f"\nNo candidates found for {ztf_id}. The object may not exist or API error.")
        continue
    if "error" in result:
        print(f"\nAPI returned error for {ztf_id}: {result.get('error')}")
        continue

    candidates = result['candidates']
    if not candidates:
        print(f"\nNo light curve data for {ztf_id}")
        continue

    output_rows = []
    for cand in candidates:
        # fid: filter ID (1=g, 2=r)
        fid = cand.get("fid")
        fid_to_letter = {1: "g", 2: "r", 3: "i"}
        filter_letter = fid_to_letter.get(fid, fid) if fid is not None else None

        out_row = {
            'ztf_id': ztf_id,
            'iauid': iauid,
            'MJD': cand.get('mjd', None),
            'filter': filter_letter,
            'unforced_mag': cand.get('magpsf', None),
            'unforced_mag_error': cand.get('sigmapsf', None),
            'unforced_mag_status': 'positive' if cand.get('isdiffpos', 't') == 't' else 'negative',
            'forced_ujy': cand.get('forcediffimflux', None),
            'forced_ujy_error': cand.get('forcediffimfluxunc', None),
        }
        output_rows.append(out_row)

    # Save per-object light curve CSV in run_folder
    df_obj = pd.DataFrame(output_rows)
    out_path = run_folder / f"{ztf_id}_lightcurve.csv"
    df_obj.to_csv(out_path, index=False)
    print(f" Saved {len(output_rows)} rows to {out_path.name}.", end=' ')
    try:
        processLasairData(df_obj, out_path)
        num_success += 1
        print(f"Done ({num_success}/{num_processed})")
    except Exception as e:
        print(f"Error running processLasairData: {e} ({num_success}/{num_processed})")

print(f"\nAll done! {num_success} of {num_total} objects processed successfully.")

Lasair client initialized.

Processing   1/423: ZTF17aabtvsy... Saved 60 rows to ZTF17aabtvsy_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF17aabtvsy_lightcurve.csv (23 rows)
Done (1/1)

Processing   2/423: ZTF17aabvong... Saved 85 rows to ZTF17aabvong_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF17aabvong_lightcurve.csv (31 rows)
Done (2/2)

Processing   3/423: ZTF17aacldgo... Saved 74 rows to ZTF17aacldgo_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF17aacldgo_lightcurve.csv (17 rows)
Done (3/3)

Processing   4/423: ZTF17aadlxmv... Saved 71 rows to ZTF17aadlxmv_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF17aadlxmv_lightcurve.csv (28 rows)
Done (4/4)

Processing   5/423: ZTF18aaaonon... Saved 105 rows to ZTF18aaaonon_lightcurve.csv. Successfully cleaned and 

Processing  39/423: ZTF18aansqov... Saved 53 rows to ZTF18aansqov_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18aansqov_lightcurve.csv (21 rows)
Done (39/39)

Processing  40/423: ZTF18aansqun... Saved 13 rows to ZTF18aansqun_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18aansqun_lightcurve.csv (13 rows)
Done (40/40)

Processing  41/423: ZTF18aanxlcr... Saved 35 rows to ZTF18aanxlcr_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18aanxlcr_lightcurve.csv (20 rows)
Done (41/41)

Processing  42/423: ZTF18aanygjk... Saved 111 rows to ZTF18aanygjk_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18aanygjk_lightcurve.csv (39 rows)
Done (42/42)

Processing  43/423: ZTF18aaoufjt... Saved 36 rows to ZTF18aaoufjt_lightcurve.csv. Successfully cleaned and wrote light curve da

Processing  77/423: ZTF18aauotcv... Saved 58 rows to ZTF18aauotcv_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18aauotcv_lightcurve.csv (26 rows)
Done (77/77)

Processing  78/423: ZTF18aavdwmh... Saved 33 rows to ZTF18aavdwmh_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18aavdwmh_lightcurve.csv (11 rows)
Done (78/78)

Processing  79/423: ZTF18aaviawx... Saved 69 rows to ZTF18aaviawx_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18aaviawx_lightcurve.csv (13 rows)
Done (79/79)

Processing  80/423: ZTF18aavrwhu... Saved 21 rows to ZTF18aavrwhu_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18aavrwhu_lightcurve.csv (21 rows)
Done (80/80)

Processing  81/423: ZTF18aavryhc... Saved 24 rows to ZTF18aavryhc_lightcurve.csv. Successfully cleaned and wrote light curve dat

Processing 115/423: ZTF18aayiahw... Saved 8 rows to ZTF18aayiahw_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18aayiahw_lightcurve.csv (8 rows)
Done (115/115)

Processing 116/423: ZTF18aayidfz... Saved 14 rows to ZTF18aayidfz_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18aayidfz_lightcurve.csv (14 rows)
Done (116/116)

Processing 117/423: ZTF18aayjvve... Saved 23 rows to ZTF18aayjvve_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18aayjvve_lightcurve.csv (23 rows)
Done (117/117)

Processing 118/423: ZTF18aayjyub... Saved 42 rows to ZTF18aayjyub_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18aayjyub_lightcurve.csv (42 rows)
Done (118/118)

Processing 119/423: ZTF18aayyqia... Saved 37 rows to ZTF18aayyqia_lightcurve.csv. Successfully cleaned and wrote light cur

Processing 153/423: ZTF18abdnffn... Saved 125 rows to ZTF18abdnffn_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18abdnffn_lightcurve.csv (48 rows)
Done (153/153)

Processing 154/423: ZTF18abdnihj... Saved 54 rows to ZTF18abdnihj_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18abdnihj_lightcurve.csv (26 rows)
Done (154/154)

Processing 155/423: ZTF18abdrmin... Saved 14 rows to ZTF18abdrmin_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18abdrmin_lightcurve.csv (14 rows)
Done (155/155)

Processing 156/423: ZTF18abeecwe... Saved 29 rows to ZTF18abeecwe_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18abeecwe_lightcurve.csv (29 rows)
Done (156/156)

Processing 157/423: ZTF18abeegsl... Saved 13 rows to ZTF18abeegsl_lightcurve.csv. Successfully cleaned and wrote light 

Processing 191/423: ZTF18abkifng... Saved 11 rows to ZTF18abkifng_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18abkifng_lightcurve.csv (11 rows)
Done (191/191)

Processing 192/423: ZTF18abktphg... Saved 15 rows to ZTF18abktphg_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18abktphg_lightcurve.csv (15 rows)
Done (192/192)

Processing 193/423: ZTF18abkudjo... Saved 17 rows to ZTF18abkudjo_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18abkudjo_lightcurve.csv (17 rows)
Done (193/193)

Processing 194/423: ZTF18ablhvfr... Saved 20 rows to ZTF18ablhvfr_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18ablhvfr_lightcurve.csv (20 rows)
Done (194/194)

Processing 195/423: ZTF18ablmflp... Saved 12 rows to ZTF18ablmflp_lightcurve.csv. Successfully cleaned and wrote light c

Processing 229/423: ZTF18abrebsb... Saved 39 rows to ZTF18abrebsb_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18abrebsb_lightcurve.csv (28 rows)
Done (229/229)

Processing 230/423: ZTF18abrfccm... Saved 145 rows to ZTF18abrfccm_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18abrfccm_lightcurve.csv (36 rows)
Done (230/230)

Processing 231/423: ZTF18abrqedj... Saved 130 rows to ZTF18abrqedj_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18abrqedj_lightcurve.csv (66 rows)
Done (231/231)

Processing 232/423: ZTF18abrzeym... Saved 23 rows to ZTF18abrzeym_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18abrzeym_lightcurve.csv (23 rows)
Done (232/232)

Processing 233/423: ZTF18absdgon... Saved 16 rows to ZTF18absdgon_lightcurve.csv. Successfully cleaned and wrote light

Processing 267/423: ZTF18acemhyb... Saved 589 rows to ZTF18acemhyb_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18acemhyb_lightcurve.csv (159 rows)
Done (267/267)

Processing 268/423: ZTF18aceyjco... Saved 15 rows to ZTF18aceyjco_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18aceyjco_lightcurve.csv (9 rows)
Done (268/268)

Processing 269/423: ZTF18aceynvm... Saved 26 rows to ZTF18aceynvm_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18aceynvm_lightcurve.csv (18 rows)
Done (269/269)

Processing 270/423: ZTF18acfwmqj... Saved 50 rows to ZTF18acfwmqj_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18acfwmqj_lightcurve.csv (23 rows)
Done (270/270)

Processing 271/423: ZTF18achxcsh... Saved 47 rows to ZTF18achxcsh_lightcurve.csv. Successfully cleaned and wrote light 

Processing 305/423: ZTF18acurlbj... Saved 26 rows to ZTF18acurlbj_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18acurlbj_lightcurve.csv (15 rows)
Done (305/305)

Processing 306/423: ZTF18acusbdy... Saved 28 rows to ZTF18acusbdy_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18acusbdy_lightcurve.csv (14 rows)
Done (306/306)

Processing 307/423: ZTF18acusbyf... Saved 42 rows to ZTF18acusbyf_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18acusbyf_lightcurve.csv (22 rows)
Done (307/307)

Processing 308/423: ZTF18acuscgc... Saved 28 rows to ZTF18acuscgc_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18acuscgc_lightcurve.csv (13 rows)
Done (308/308)

Processing 309/423: ZTF18acuscrr... Saved 33 rows to ZTF18acuscrr_lightcurve.csv. Successfully cleaned and wrote light c